In [27]:
def avg(df):
    df['Avg'] = df['Turnover']/df['Volume']
    df['Avg'].fillna(df['Close'], inplace=True)

In [28]:
from tuneta.tune_ta import TuneTA
import pandas as pd
import numpy as np

import talib

PATH_READ_TRAIN = '123181_train_raw.csv'
PATH_READ_TEST = '123181_test_raw.csv'

df = pd.read_csv(PATH_READ_TRAIN, index_col='Timestamp')
df_test = pd.read_csv(PATH_READ_TEST, index_col='Timestamp')

df = df.drop(['InstrumentID','TradingDay','PreClosePrice',], axis=1)
df.index = pd.to_datetime(df.index)

# df['Avg'] = talib.AVGPRICE(df['Open'], df['High'], df['Low'], df['Close'])
# df_test['Avg'] = talib.AVGPRICE(df_test['Open'], df_test['High'], df_test['Low'], df_test['Close'])

avg(df)
avg(df_test)

df.to_csv('Temp.csv')

print(df['Avg'].info())


PATH_STOCK_TRAIN = '301085_train_raw.csv'
PATH_STOCK_TEST = '301085_test_raw.csv'

stock = pd.read_csv(PATH_STOCK_TRAIN, index_col= 'Timestamp')
stock_test = pd.read_csv(PATH_STOCK_TEST, index_col= 'Timestamp')

stock = stock.drop(['InstrumentID','TradingDay','PreClosePrice',], axis=1)
stock.index = pd.to_datetime(stock.index)

# stock['Avg'] = talib.AVGPRICE(stock['Open'], stock['High'], stock['Low'], stock['Close'])
# stock_test['Avg'] = talib.AVGPRICE(stock_test['Open'], stock_test['High'], stock_test['Low'], stock_test['Close'])

avg(stock)
avg(stock_test)

print(stock['Avg'].info())


print(df.columns)

<class 'pandas.core.series.Series'>
DatetimeIndex: 5840 entries, 2023-06-01 09:26:00 to 2023-07-06 15:35:00
Series name: Avg
Non-Null Count  Dtype  
--------------  -----  
5840 non-null   float64
dtypes: float64(1)
memory usage: 91.2 KB
None
<class 'pandas.core.series.Series'>
DatetimeIndex: 5839 entries, 2023-06-01 09:26:00 to 2023-07-06 15:35:00
Series name: Avg
Non-Null Count  Dtype  
--------------  -----  
5839 non-null   float64
dtypes: float64(1)
memory usage: 91.2 KB
None
Index(['Open', 'High', 'Low', 'Volume', 'Turnover', 'Close', 'Avg'], dtype='object')


In [29]:
def target(df, column, period):
    if period == 1:
        return (df[column].shift(-1) - df[column])/df[column]
    if period > 1:
        return (df[column].rolling(window=period).mean().shift(-1 * period) - df[column]) / df[column]

In [30]:
def class_target(value):
    if value > 0.001:
        return 1
    elif value < -0.001:
        return -1
    else:
        return 0

In [31]:
df['Target_Close_1min'] = target(df,'Close', 1)
df['Target_Avg_1min'] = target(df, 'Avg', 1)

df = df[df['Target_Close_1min'] != 0]
df = df.dropna()

df_test['Target_Close_1min'] = target(df_test,'Close', 1)
df_test['Target_Avg_1min'] = target(df_test, 'Avg', 1)

df_test = df_test[df_test['Target_Close_1min'] != 0]
df_test = df_test.dropna()

In [32]:
train_X = df[['Open',
            'High',
            'Low',
            'Volume',
            'Turnover',
            'Close']]

test_X = df_test[['Open',
            'High',
            'Low',
            'Volume',
            'Turnover',
            'Close']]

train_y = df['Target_Avg_1min']
test_y = df_test['Target_Avg_1min']

In [33]:
tt = TuneTA(n_jobs=8, verbose=True)

tt.fit(train_X, train_y,
# 优化指标
indicators=['tta'],
# 待优化参数的两个参数范围（时间的短期和长期）
ranges=[(4, 40)],
# 每个时间段最多100次试验，以搜索最佳指标参数
trials=300,
# 在每个时间段持续20次试验没有改善后停止搜索参数
early_stop=50)

In [34]:
tt.prune(max_inter_correlation=.7)

tt.report(target_corr=True, features_corr=True)


Indicator Correlation to Target:

                                                             Correlation
---------------------------------------------------------  -------------
tta_BOP                                                         0.371416
tta_NATR_timeperiod_7                                           0.304935
tta_STOCHRSI_timeperiod_19_fastk_period_4_fastd_period_11       0.278148
tta_CDLCLOSINGMARUBOZU                                          0.22082
tta_MACD_fastperiod_33_slowperiod_4_signalperiod_37             0.214914
tta_MINUS_DM_timeperiod_7                                       0.200023
tta_ULTOSC_timeperiod1_38_timeperiod2_6_timeperiod3_27          0.198291
tta_LINEARREG_SLOPE_timeperiod_9                                0.188017
tta_CDLBELTHOLD                                                 0.175156
tta_DX_timeperiod_26                                            0.156689
tta_HT_PHASOR                                                   0.156649
tta_MFI_timeperio

In [35]:
features_train = tt.transform(train_X)

features_test = tt.transform(test_X)

Now training stock

In [36]:
stock = stock.merge(df['Target_Avg_1min'], how='right',left_index=True,right_index=True)
stock_test = stock_test.merge(df_test['Target_Avg_1min'], how='right',left_index=True,right_index=True)

In [37]:
stock = stock.dropna()
stock_test = stock_test.dropna()

In [38]:
STOCK_train_X = stock[['Open',
            'High',
            'Low',
            'Volume',
            'Turnover',
            'Close']]

STOCK_test_X = stock_test[['Open',
            'High',
            'Low',
            'Volume',
            'Turnover',
            'Close']]

STOCK_train_y = stock['Target_Avg_1min']
STOCK_test_y = stock_test['Target_Avg_1min']

tt = TuneTA(n_jobs=8, verbose=True)

tt.fit(STOCK_train_X, STOCK_train_y,
# 优化指标
indicators=['tta'],
# 待优化参数的两个参数范围（时间的短期和长期）
ranges=[(4, 40)],
# 每个时间段最多100次试验，以搜索最佳指标参数
trials=300,
# 在每个时间段持续20次试验没有改善后停止搜索参数
early_stop=50)

tt.prune(max_inter_correlation=.7)

tt.report(target_corr=True, features_corr=True)


Indicator Correlation to Target:

                                                           Correlation
-------------------------------------------------------  -------------
tta_STDDEV_timeperiod_9                                       0.231649
tta_TRANGE                                                    0.212807
tta_BOP                                                       0.206839
tta_MACD_fastperiod_33_slowperiod_4_signalperiod_37           0.178009
tta_ADOSC_fastperiod_14_slowperiod_6                          0.167606
tta_STOCHF_fastk_period_4_fastd_period_36                     0.159744
tta_OBV                                                       0.146839
tta_PLUS_DI_timeperiod_36                                     0.137284
tta_CDLCLOSINGMARUBOZU                                        0.135566
tta_HT_PHASOR                                                 0.133435
tta_WILLR_timeperiod_16                                       0.124244
tta_ULTOSC_timeperiod1_34_timeperiod2_26_t

In [39]:
STOCK_features_train = tt.transform(STOCK_train_X)

STOCK_features_test = tt.transform(STOCK_test_X)

Renaming STOCK features

In [40]:
STOCK_features_train = STOCK_features_train.add_prefix('STOCK_')
STOCK_features_test = STOCK_features_test.add_prefix('STOCK_')

In [41]:
print(STOCK_features_train.shape)
print(features_train.shape)

(5476, 75)
(5478, 66)


In [42]:
print('Number of rows before cleaning: %d'%features_train.shape[0])
features_train = features_train.dropna()
print('Number of rows after cleaning: %d'%features_train.shape[0])


print('Number of rows before cleaning: %d'%features_test.shape[0])
features_test = features_test.dropna()
print('Number of rows after cleaning: %d'%features_test.shape[0])

Number of rows before cleaning: 5478
Number of rows after cleaning: 5410
Number of rows before cleaning: 2382
Number of rows after cleaning: 2314


In [43]:
merged_train = features_train.merge(STOCK_features_train, how='left',left_index=True,right_index=True)
merged_test = features_test.merge(STOCK_features_test, how='left',left_index=True,right_index=True)

Clean after merging

In [44]:
print('Number of rows before cleaning: %d'%merged_train.shape[0])
merged_train = merged_train.dropna()
print('Number of rows after cleaning: %d'%merged_train.shape[0])


print('Number of rows before cleaning: %d'%merged_test.shape[0])
merged_test = merged_test.dropna()
print('Number of rows after cleaning: %d'%merged_test.shape[0])

Number of rows before cleaning: 5410
Number of rows after cleaning: 5408
Number of rows before cleaning: 2314
Number of rows after cleaning: 2314


In [45]:
print(merged_train.columns)

Index(['tta_BOP', 'tta_NATR_timeperiod_7',
       'tta_STOCHRSI_timeperiod_19_fastk_period_4_fastd_period_11_0',
       'tta_STOCHRSI_timeperiod_19_fastk_period_4_fastd_period_11_1',
       'tta_CDLCLOSINGMARUBOZU',
       'tta_MACD_fastperiod_33_slowperiod_4_signalperiod_37_0',
       'tta_MACD_fastperiod_33_slowperiod_4_signalperiod_37_1',
       'tta_MACD_fastperiod_33_slowperiod_4_signalperiod_37_2',
       'tta_MINUS_DM_timeperiod_7',
       'tta_ULTOSC_timeperiod1_38_timeperiod2_6_timeperiod3_27',
       ...
       'STOCK_tta_CDLHARAMI', 'STOCK_tta_CDLSEPARATINGLINES',
       'STOCK_tta_CDLHANGINGMAN', 'STOCK_tta_CDL3INSIDE',
       'STOCK_tta_CDLDOJISTAR', 'STOCK_tta_CDLSHOOTINGSTAR',
       'STOCK_tta_CDLHARAMICROSS', 'STOCK_tta_CDLTHRUSTING',
       'STOCK_tta_CDLHOMINGPIGEON', 'STOCK_tta_CDLSTALLEDPATTERN'],
      dtype='object', length=141)


In [46]:
train_result = pd.merge(merged_train, train_y, how='left', left_index= True, right_index=True)
test_result = pd.merge(merged_test, test_y, how='left', left_index= True, right_index=True)

In [47]:
train_result.to_csv('train_withF_VT_Avg_1min_FULL.csv')
test_result.to_csv('test_withF_VT_Avg_1min_FULL.csv')